Analisís inicial del JSON. Antes de iniciar descargue el archivo y con CTRL+ F encontré la propiedad "custom_attributes" indicada en el
ejercicio. Manualmente también me aseguré de que fuese un solo producto por medio del SKU.

Despues le pedí a GPT un script para buscar esa propiedad y que me imprima el path en formato de Python.

In [32]:
import json
import requests

url = "https://storage.googleapis.com/resources-prod-shelftia/scrapers-prueba/product.json"
output_file = "product.json"

response = requests.get(url)
data = response.json()

matches = []

# Este script es solo para yo entender mejor la estructura del JSON, como tal no es necesario en la implementación final.
def recurse(obj, path="data"):
    if isinstance(obj, dict):
        if obj.get("name") == "custom_attributes":
            value = obj.get("value", {})
            for lang_key in ("en-CR", "es-CR"):
                if lang_key in value:
                    try:
                        parsed = json.loads(value[lang_key])
                        matches.append(parsed)
                        # Se imprime en formato de python.
                        print(f"{path}['value']['{lang_key}']")
                    except json.JSONDecodeError:
                        pass
        for k, v in obj.items():
            recurse(v, f"{path}['{k}']")
    elif isinstance(obj, list):
        for idx, item in enumerate(obj):
            recurse(item, f"{path}[{idx}]")

recurse(data)            

with open(output_file, 'w', encoding='utf-8', newline='\n') as f:
    json.dump(data, f, indent=2, ensure_ascii=False)


data['allVariants'][0]['attributesRaw'][31]['value']['en-CR']
data['allVariants'][0]['attributesRaw'][31]['value']['es-CR']


Se verifica que el path es valido y contiene un JSON en formato de string. Se utiliza la función next ya que e otras ocasiones
me ha sucedido que no necesariamente la propiedad que se busca se localiza en el indice 31. Next es mas robusto en caso
de que se quisieran analizar multiples productos y que custom_attributes no se encuentre en el índice 31

In [ ]:
custom_attr_item = next(
    (item for item in data['allVariants'][0]['attributesRaw'] if item.get("name") == "custom_attributes"),
    None
)

custom_props_str = custom_attr_item['value']['en-CR'] if custom_attr_item else ''
custom_props_json = json.loads(custom_props_str) if custom_props_str else {}
print(custom_props_str)


{"storage":{"value":{"code":"refrigerated","name":"Refrigerated"},"name":"Storage","type":"pim_catalog_simpleselect"},"country_of_origin":{"value":{"code":"united_states","name":"United States"},"name":"Country of Origin","type":"pim_catalog_simpleselect"},"imported_national":{"value":{"code":"imported","name":"Imported"},"name":"Imported / National","type":"pim_catalog_simpleselect"},"form":{"value":{"code":"block","name":"Block"},"name":"Form","type":"pim_catalog_simpleselect"},"allergens":{"value":[{"code":"Milk","name":"Milk"}],"name":"Allergens","type":"pim_catalog_multiselect"},"sku":{"value":"29041","name":"SKU (Item Number)","type":"pim_catalog_identifier"},"show_online":{"value":true,"name":"Show Online","type":"pim_catalog_boolean"},"add_to_cart":{"value":true,"name":"Add To Cart at Search/category","type":"pim_catalog_boolean"},"vegan":{"value":false,"name":"Vegan","type":"pim_catalog_boolean"},"kosher":{"value":false,"name":"Kosher","type":"pim_catalog_boolean"},"organic":{

Le proporciono a la IA los valores que quiero extraer de este JSON. Le indico que utlize la librería de pandas.
Con el codígo proporcionado verifico que maneje los edge cases como propiedades no existentes o propiedades de
tipo lista. En este caso si las manejo correctamente. Las listas aparecen como strings con comas y 
si alguna propiedad no existe como un string vacio.

Adicional agregue una función de utilidad para trabajar con datos en español.


In [55]:
import pandas as pd
import json

langs = ['en-CR', 'es-CR']

keys = [
    "Allergens", "SKU", "Vegan", "Kosher", "Organic",
    "Vegetarian", "Gluten_free", "Lactose_free",
    "Package_quantity", "Unit_size", "Net_weight",
    "Propiedad_no_existente"
]

def translate_boolean(value : bool) -> str:
    return "Verdadero" if value is True else "Falso"

def translate_lang(lang_code: str) -> str:
    return {
        "en-CR": "English (Costa Rica)",
        "es-CR": "Español (Costa Rica)"
    }.get(lang_code, lang_code)

rows = []

for lang in langs:
    props_str = custom_attr_item['value'].get(lang, '') if custom_attr_item else ''
    props_json = json.loads(props_str) if props_str else {}
    
    row = {"Language": translate_lang(lang)}

    for key in keys:
        val = props_json.get(key.lower(), {}).get("value", "")
        if isinstance(val, list):
            val = ", ".join([item.get("name", "") for item in val])

        # Usar traducciones en español en caso de ser necesario.
        elif lang == 'es-CR' and isinstance(val, bool):
            val = translate_boolean(val)

        row[key] = val

    rows.append(row)


# Create and save CSV
df = pd.DataFrame(rows)
df.to_csv("custom_props_bilingual.csv", index=False)

JupyterNotebook permite visualizar tablas limpiamente. Me apoye de IA para que se vea centrado, esto no afecta al CSV solo a la 
visualización.

In [57]:
from IPython.display import display
# Center all cells using Styler
styled_df = df.style.set_table_styles(
    [{'selector': 'td', 'props': [('text-align', 'center')]},
     {'selector': 'th', 'props': [('text-align', 'center')]}]
).set_properties(**{'text-align': 'center'}).hide(axis="index")

display(styled_df)

Language,Allergens,SKU,Vegan,Kosher,Organic,Vegetarian,Gluten_free,Lactose_free,Package_quantity,Unit_size,Net_weight,Propiedad_no_existente
English (Costa Rica),Milk,29041,False,False,False,True,True,False,1.000000,452.0000,452.0000,
Español (Costa Rica),Leche,29041,Falso,Falso,Falso,Verdadero,Verdadero,Falso,1.000000,452.0000,452.0000,
